In [2]:
import pandas as pd
import numpy as np

from pyomo.environ import *

from data_cleaning import hub_occ_dict
from data_cleaning import heatdays_df
from data_cleaning import cengeo_pop_dict
from data_cleaning import bg_ces_df

dist_to_hub_df = pd.read_csv('data/distmatrix_contracosta.csv')
dist_to_hub_df.set_index('Unnamed: 0', inplace = True)
dist_to_hub_df.index.name = None

ImportError: cannot import name 'hub_occ_dict' from 'data_cleaning' (C:\Users\18313\Desktop\sgc_resilience\4_deployment\sgc-deployment-scenarios\data_cleaning.py)

### Define Model

_Create model and variable indices_

In [ ]:
model = ConcreteModel()

model.init_cengeos = Set(initialize = dist_to_hub_df.index)
model.init_hubs = Set(initialize = dist_to_hub_df.columns)

_Create set of blockgroup, hub pairs within five miles of each other_

In [ ]:
def filter_to_nearby_hubs(model, cg, hub):
    return not np.isnan(dist_to_hub_df.loc[cg, hub])

model.idx_cg_hub_pairs = Set(initialize = model.init_cengeos*model.init_hubs, filter = filter_to_nearby_hubs)

In [ ]:
included_cengeos = []
included_hubs = []

for pair in model.idx_cg_hub_pairs:
    if pair[0] not in included_cengeos:
        included_cengeos.append(pair[0])
    if pair[1] not in included_hubs:
        included_hubs.append(pair[1])
        
model.idx_cengeos = Set(initialize = included_cengeos)
model.idx_hubs = Set(initialize = included_hubs)

### Define Parameters

_Pairwise distance indexed on (blockgroup, hub) tuple_

In [ ]:
def get_cg_hub_distance(model, cg, hub):
    return dist_to_hub_df.loc[cg, hub]

model.param_cg_hub_dist = Param(model.idx_cg_hub_pairs, initialize = get_cg_hub_distance)

_Dictionary of blockgroups within range for each hub_

In [ ]:
cgs_at_hub = {hub: [] for hub in model.idx_hubs}

for hub in cgs_at_hub:
    for pair in model.idx_cg_hub_pairs:
        if pair[1] == hub:
            cgs_at_hub[hub].append(pair[0])

hubs_with_no_cgs = [key for key in cgs_at_hub.keys() if len(cgs_at_hub[key]) == 0]

for hub in hubs_with_no_cgs:
    del cgs_at_hub[hub]
    
model.param_cgs_at_hub = Param(model.idx_hubs, within=Any, initialize=cgs_at_hub)

### Define Variables

_Is this site a hub?_

In [ ]:
model.var_hub_yn = Var(model.idx_hubs, initialize = 1, within = Binary)

_What proportion of a blockgroup's population is served by this hub?_

In [ ]:
model.var_prop_cg_at_hub = Var(model.idx_cengeos, model.idx_hubs, initialize = 0.0, bounds = (0.0, 1.0))

_How many people are served by this hub?_

In [ ]:
def ppl_at_hub(model, hub):
    return(sum(model.var_prop_cg_at_hub[cg, hub].value*cengeo_pop_dict[cg] for cg in model.param_cgs_at_hub[hub]))

model.var_hub_pop = Var(model.idx_hubs, initialize = ppl_at_hub, domain = NonNegativeReals)

def ppl_served_at_hub(model, hub):
    tot_ppl_served = 0

    for cg in model.idx_cengeos:
        if (cg, hub) in model.idx_cg_hub_pairs:
            ppl_served = model.var_prop_cg_at_hub[cg, hub]*cengeo_pop_dict[cg]
            tot_ppl_served = ppl_served + tot_ppl_served
        else:
            tot_ppl_served = tot_ppl_served
    
    return(tot_ppl_served)

model.var_hub_pop = Var(model.idx_hubs, initialize = ppl_served_at_hub, domain = NonNegativeReals)

_How many people in this block group are assigned to a hub?_

# del
def cg_total_assigned(model, cg):
    tot_ppl_served = 0

    for hub in model.idx_hubs:
        if (cg, hub) in model.idx_cg_hub_pairs:
            ppl_served = model.var_prop_cg_at_hub[cg, hub]*cengeo_pop_dict[cg]
            tot_ppl_served = ppl_served + tot_ppl_served
        else:
            tot_ppl_served = tot_ppl_served
    
    return(tot_ppl_served)

model.var_cg_pop_assigned = Var(model.idx_cengeos, initialize = cg_total_assigned, domain = NonNegativeReals)

### Define Objective

_Minimize aggregate travel distance_

In [ ]:
min_agg_dist = sum(model.param_cg_hub_dist[cg, hub] * cengeo_pop_dict[cg] * model.var_hub_pop[hub] 
                   for cg, hub in model.idx_cg_hub_pairs)

model.obj_min_agg_dist = Objective(expr = min_agg_dist, sense = minimize)

### Define Constraints

_Hub cannot serve more people than it has capacity for, but must be mostly full_

In [ ]:
def serve_less_than_hub_occ(model, hub):
    return(model.var_hub_pop[hub] <= hub_occ_dict[hub]*model.var_hub_yn[hub])

model.con_max_occ = Constraint(model.idx_hubs, rule = serve_less_than_hub_occ)

_Construct a set number of hubs_

In [ ]:
model.con_max_hubs = Constraint(expr = sum(model.var_hub_yn[hub] for hub in model.idx_hubs) == 3)

_No one is served at a location if it is not deemed a hub_

# del
def open_only(model, cg, hub):
    return(model.var_prop_cg_at_hub[cg, hub] <= model.var_hub_yn[hub])

model.con_open_only = Constraint(model.idx_cg_hub_pairs, rule = open_only)

_Meet a certain portion of demand in area of interest_

In [ ]:
max_service_pop = sum(cengeo_pop_dict[cg] for cg in model.idx_cengeos)    
min_service_pop = 0.05*max_service_pop

service_pop = sum(model.var_hub_pop[hub] for hub in model.idx_hubs)

model.con_min_coverage = Constraint(expr = (min_service_pop, service_pop, max_service_pop))

_Prioritize CalEnviroScreen populations_

model.con_serve_ces = ConstraintList()

for cg in model.idx_cengeos:
    cg_ces_score = float(bg_ces_df.loc[bg_ces_df['GISJOIN'] == cg, 'SCORE_PCTL_CI_BG'])

    if cg_ces_score >= 75.0:
        model.con_serve_ces.add(expr = model.var_cg_pop_assigned[cg] >= 0.95*cengeo_pop_dict[cg])

_Do not meet more than 100% of demand_

model.con_demand_max = ConstraintList() #Constrain the TOTAL proportion of people assigned a hub to be 100 percent

for cg in model.idx_cengeos:
    # Only add constraint for cengeos that have hubs, may return error if "if" statement ommited
    if sum([pair[0]==cg for pair in model.idx_cg_hub_pairs]) >= 1:
        model.con_demand_max.add(expr = sum(model.var_prop_cg_at_hub[pair] for pair in model.idx_cg_hub_pairs if pair[0]==cg)<=1)

### Solve Model

In [ ]:
import pyomo.opt as pyopt

In [ ]:
pyopt.SolverFactory('glpk').solve(model)

In [ ]:
model.var_prop_cg_at_hub.pprint()

In [ ]:
for hub in model.idx_hubs:
    print(hub)
    if model.var_hub_yn[hub].value == 1:
        for cg in model.param_cgs_at_hub[hub]:
            print(cg, model.var_prop_cg_at_hub[cg, hub].value, cengeo_pop_dict[cg])

In [ ]:
model.var_hub_pop.pprint()

In [ ]:
model.var_hub_yn.pprint()

In [ ]:
res_hub_pop = [model.var_hub_pop[hub].value for hub in model.idx_hubs]
res_hub_yn = [model.var_hub_yn[hub].value for hub in model.idx_hubs]

res_prop_cg_at_hub_list = []

for cg in model.idx_cengeos:
    cg_dict = dict()
    for pair in model.idx_cg_hub_pairs:
        if pair[0]==cg:
            cg_dict[pair[1]] = model.var_prop_cg_at_hub[pair].value
    res_prop_cg_at_hub_list.append(cg_dict)
    
res_prop_cg_at_hub = pd.DataFrame(res_prop_cg_at_hub_list)

In [ ]:
hub_y = [hub for hub in model.var_hub_yn if model.var_hub_yn[hub].value > 0]

for hub in hub_y:
    print(hub, np.unique(res_prop_cg_at_hub[hub]))
    print(hub, model.var_hub_pop[hub].value)

In [ ]:
np.unique(res_prop_cg_at_hub)

model_solver = pyopt.SolverFactory('glpk')

model_results = model_solver.solve(model, tee=True)

results_df = pd.DataFrame(index = pd.MultiIndex.from_tuples(model.idx_cg_hub_pairs, names = ['cengeos', 'hubs']))